In [ ]:
import pandas as pd
import numpy as np

SyntaxError: invalid syntax (2774096220.py, line 4)

In [12]:
df_ref = pd.read_csv("reference_key.csv")
display(df_ref.head())
df_ref.shape

,Company,Company Key,Cleaned Company,Parent Company
0,Lion Global Investors Ltd,lion global investors ltd,Lion Global Investors,OCBC Bank
1,Credit Suisse Securities (Singapore) Ltd.,credit suisse securities (singapore) ltd.,Credit Suisse Securities (Singapore),UBS Group
2,"Forward Management, LLC","forward management, llc",Forward Management,Salient Partners
3,Morgan Stanley Investment Management,morgan stanley investment management,Morgan Stanley Investment Management,Morgan Stanley Investment Management
4,Rockefeller Asset Management,rockefeller asset management,Rockefeller Asset Management,Rockefeller Asset Management


(841, 4)

In [40]:
df = pd.read_csv('test_HD.csv')
df = df[["\nName","Name","Cleaned Company", "Parent", "Nationality"]]
display(df.head())
df.shape

/var/folders/gt/3sb0875538n39l60gwltp88w0000gn/T/ipykernel_41938/3210950467.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('test_HD.csv')


,\nName,Name,Cleaned Company,Parent,Nationality
0,NaN,"Indorama Resources Co., ltd.","Indorama Resources Co., ltd.","Indorama Resources Co., ltd.",MAURITIAN
1,NaN,Thai NVDR Company Limited,Thai NVDR Company Limited,Thai NVDR Company Limited,THAI
2,NaN,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,THAI
3,NaN,Social Security Office,Social Security Office,Ministry of Labour Thailand,THAI
4,NaN,Mutual Fund Vayupak One by MFC Asset Managemen...,Mutual Fund Vayupak One by MFC Asset Managemen...,Mutual Fund Vayupak One by MFC Asset Managemen...,THAI


(39799, 5)

In [48]:
df["Company Key"] = df["Cleaned Company"].str.strip().str.lower()
df["Matched"] = df["Cleaned Company"].isin(df_ref["Company"])
df["Matched"].value_counts()

Matched
False    39795
True         4
Name: count, dtype: int64

### NUMBER OF VALUES MATCHED: 4 -> 4/39799

In [52]:
df.loc[df["\nName"].isna(), "Name"].isin(df_ref["Company"]).value_counts()

Name
False    942
True       4
Name: count, dtype: int64

### NUMBER OF COMPANIES MATCHED: 4 -> 4/946

In [59]:
df.loc[df["\nName"].isna(), "Name"].str.strip().str.lower().isin(df_ref["Company Key"]).value_counts()

Name
False    938
True       8
Name: count, dtype: int64

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


### NUMBER OF COMPANIES MATCHED:8 -> 8/946

In [70]:
from thefuzz import fuzz
from thefuzz import process
import pandas as pd

# Create dictionary for lookup
ref_dict = dict(zip(df_ref["Company Key"].astype(str), df_ref["Parent Company"]))

# Safe fuzzy matcher
def get_best_match(company_key):
    if pd.isna(company_key):
        return pd.Series([None, None, None])
    
    company_key = str(company_key)
    match, score = process.extractOne(company_key, ref_dict.keys(), scorer=fuzz.token_sort_ratio)
    parent_company = ref_dict[match] if score > 70 else None
    return pd.Series([match, score, parent_company])

# Apply
df[["Best Match", "Match Score", "Parent Company"]] = df["Company Key"].apply(get_best_match)

In [78]:
df

,\nName,Name,Cleaned Company,Parent,Nationality,Matched Company,Company Key,Matched,Best Match,Match Score,Parent Company
0,NaN,"Indorama Resources Co., ltd.","Indorama Resources Co., ltd.","Indorama Resources Co., ltd.",MAURITIAN,NaN,"indorama resources co., ltd.",False,"daiwa securities co., ltd.",67.0,None
1,NaN,Thai NVDR Company Limited,Thai NVDR Company Limited,Thai NVDR Company Limited,THAI,NaN,thai nvdr company limited,False,tisco securities company limited,70.0,None
2,NaN,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,THAI,NaN,bangkok bank public company limited,False,bangkok insurance public company limited,80.0,Bangkok Insurance
3,NaN,Social Security Office,Social Security Office,Ministry of Labour Thailand,THAI,NaN,social security office,True,social security office,100.0,Ministry of Labour Thailand
4,NaN,Mutual Fund Vayupak One by MFC Asset Managemen...,Mutual Fund Vayupak One by MFC Asset Managemen...,Mutual Fund Vayupak One by MFC Asset Managemen...,THAI,NaN,mutual fund vayupak one by mfc asset managemen...,False,meiji yasuda asset management company ltd.,65.0,None
...,...,...,...,...,...,...,...,...,...,...,...
39794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,NaN,None
39795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,NaN,None
39796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,NaN,None
39797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,None,NaN,None


In [80]:
df[df["Match Score"] > 70.0]

,\nName,Name,Cleaned Company,Parent,Nationality,Matched Company,Company Key,Matched,Best Match,Match Score,Parent Company
2,NaN,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,Bangkok Bank Public Company Limited,THAI,NaN,bangkok bank public company limited,False,bangkok insurance public company limited,80.0,Bangkok Insurance
3,NaN,Social Security Office,Social Security Office,Ministry of Labour Thailand,THAI,NaN,social security office,True,social security office,100.0,Ministry of Labour Thailand
15,NaN,Kiatnakin Phatra Securities Public Company Lim...,Kiatnakin Phatra Securities Public Company Lim...,Kiatnakin Phatra Securities Public Company Lim...,THAI,NaN,kiatnakin phatra securities public company lim...,False,krungsri securities public company limited,80.0,Krungsri Securities
17,NaN,Thai Life Insurance Public Company Limited,Thai Life Insurance Public Company Limited,Thai Life Insurance Public Company Limited,THAI,NaN,thai life insurance public company limited,False,fwd life insurance public company limited,89.0,FWD Group
32,NaN,Bualuang Securities Public Company Limited For...,Bualuang Securities Public Company Limited For...,Bualuang Securities Public Company Limited For...,THAI,NaN,bualuang securities public company limited for...,False,bualuang securities public company limited,91.0,Bualuang Securities
...,...,...,...,...,...,...,...,...,...,...,...
39458,NaN,LGT BANK (SINGAPORE) LTD,LGT BANK (SINGAPORE) LTD,LGT BANK (SINGAPORE) LTD,NaN,NaN,lgt bank (singapore) ltd,False,citibank singapore ltd,82.0,Citigroup
39471,NaN,BANK OF SINGAPORE LIMITED,BANK OF SINGAPORE LIMITED,BANK OF SINGAPORE LIMITED,SINGAPOREAN,NaN,bank of singapore limited,False,hsbc bank (singapore) limited,85.0,HSBC Holdings
39493,NaN,CGS INTERNATIONAL SECURITIES PTE. LTD.,CGS INTERNATIONAL SECURITIES PTE. LTD.,CGS INTERNATIONAL SECURITIES PTE. LTD.,NaN,NaN,cgs international securities pte. ltd.,False,seatown holdings international pte ltd,76.0,Temasek Holdings
39495,NaN,DBS VICKERS SECURITIES (SINGAPORE) PTE LTD,DBS VICKERS SECURITIES (SINGAPORE) PTE LTD,DBS VICKERS SECURITIES (SINGAPORE) PTE LTD,SINGAPOREAN,NaN,dbs vickers securities (singapore) pte ltd,False,dbs vickers securities (singapore) pte ltd.,100.0,DBS Group
